# **Data Cleaning using SQL**

## **<u>Skills used</u>**

- Change Datatypes
- Add derived columns
- Update columns with null values with default values
- Window functions
- Type casting
- Aggregation
- String splits
- Dealing with missing data
- Case Statements
- Deleting duplicate rows
- Deleting unwanted columns

## <u>**Sections**</u>

1. Data Exploration
2. Backup and Restore
3. Datatype Updates
4. Data Cleaning/Imputation

## **Data Exploration**

In [2]:
-- View the first few rows of the table
select top 10 * from NashvilleHousing;

(10 rows affected)

Total execution time: 00:00:00.031

UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,OwnerName,OwnerAddress,Acreage,TaxDistrict,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath
2045,007 00 0 125.00,SINGLE FAMILY,"1808 FOX CHASE DR, GOODLETTSVILLE",2013-04-09,240000,20130412-0036474,No,"FRAZIER, CYRENTHA LYNETTE","1808 FOX CHASE DR, GOODLETTSVILLE, TN",2.3,GENERAL SERVICES DISTRICT,50000,168200,235700,1986,3,3,0
16918,007 00 0 130.00,SINGLE FAMILY,"1832 FOX CHASE DR, GOODLETTSVILLE",2014-06-10,366000,20140619-0053768,No,"BONER, CHARLES & LESLIE","1832 FOX CHASE DR, GOODLETTSVILLE, TN",3.5,GENERAL SERVICES DISTRICT,50000,264100,319000,1998,3,3,2
54582,007 00 0 138.00,SINGLE FAMILY,"1864 FOX CHASE DR, GOODLETTSVILLE",2016-09-26,435000,20160927-0101718,No,"WILSON, JAMES E. & JOANNE","1864 FOX CHASE DR, GOODLETTSVILLE, TN",2.9,GENERAL SERVICES DISTRICT,50000,216200,298000,1987,4,3,0
43070,007 00 0 143.00,SINGLE FAMILY,"1853 FOX CHASE DR, GOODLETTSVILLE",2016-01-29,255000,20160129-0008913,No,"BAKER, JAY K. & SUSAN E.","1853 FOX CHASE DR, GOODLETTSVILLE, TN",2.6,GENERAL SERVICES DISTRICT,50000,147300,197300,1985,3,3,0
22714,007 00 0 149.00,SINGLE FAMILY,"1829 FOX CHASE DR, GOODLETTSVILLE",2014-10-10,278000,20141015-0095255,No,"POST, CHRISTOPHER M. & SAMANTHA C.","1829 FOX CHASE DR, GOODLETTSVILLE, TN",2,GENERAL SERVICES DISTRICT,50000,152300,202300,1984,4,3,0
18367,007 00 0 151.00,SINGLE FAMILY,"1821 FOX CHASE DR, GOODLETTSVILLE",2014-07-16,267000,20140718-0063802,No,"FIELDS, KAREN L. & BRENT A.","1821 FOX CHASE DR, GOODLETTSVILLE, TN",2,GENERAL SERVICES DISTRICT,50000,190400,259800,1980,3,3,0
19804,007 14 0 002.00,SINGLE FAMILY,"2005 SADIE LN, GOODLETTSVILLE",2014-08-28,171000,20140903-0080214,No,"HINTON, MICHAEL R. & CYNTHIA M. MOORE","2005 SADIE LN, GOODLETTSVILLE, TN",1.03,GENERAL SERVICES DISTRICT,40000,137900,177900,1976,3,2,0
54583,007 14 0 024.00,SINGLE FAMILY,"1917 GRACELAND DR, GOODLETTSVILLE",2016-09-27,262000,20161005-0105441,No,"BAILOR, DARRELL & TAMMY","1917 GRACELAND DR, GOODLETTSVILLE, TN",1.03,GENERAL SERVICES DISTRICT,40000,157900,197900,1978,3,2,0
36500,007 14 0 026.00,SINGLE FAMILY,"1428 SPRINGFIELD HWY, GOODLETTSVILLE",2015-08-14,285000,20150819-0083440,No,"ROBERTS, MISTY L. & ROBERT M.","1428 SPRINGFIELD HWY, GOODLETTSVILLE, TN",1.67,GENERAL SERVICES DISTRICT,45400,176900,222300,2000,3,2,1
19805,007 14 0 034.00,SINGLE FAMILY,"1420 SPRINGFIELD HWY, GOODLETTSVILLE",2014-08-29,340000,20140909-0082348,No,"LEE, JEFFREY & NANCY","1420 SPRINGFIELD HWY, GOODLETTSVILLE, TN",1.3,GENERAL SERVICES DISTRICT,40000,179600,219600,1995,5,3,0


## **BackUp And Restore Section**

In [25]:
-- Backing up data before updating
-- Drop a table called 'NashvilleHousingbkp' in schema 'dbo'
-- Drop the table if it already exists
IF OBJECT_ID('[dbo].[NashvilleHousingbkp]', 'U') IS NOT NULL
DROP TABLE [dbo].[NashvilleHousingbkp]
GO
Select * into NashvilleHousingbkp 
from NashvilleHousing;

(56477 rows affected)

Total execution time: 00:00:00.199

In [61]:
-- Cell to restore backup if required
-- Drop a table called 'NashvilleHousing' in schema 'dbo'
-- Drop the table if it already exists
IF OBJECT_ID('[dbo].[NashvilleHousing]', 'U') IS NOT NULL
DROP TABLE [dbo].[NashvilleHousing]
GO
Select * into NashvilleHousing 
from NashvilleHousingbkp
GO

Commands completed successfully.

(56477 rows affected)

Total execution time: 00:00:00.092

## **Datatype Updates**

In [62]:
-- Checking SalePrice column
select top 5 SalePrice, convert(bigint, SalePrice) as SalePriceConverted
from NashvilleHousing;

(5 rows affected)

Total execution time: 00:00:00.013

SalePrice,SalePriceConverted
45000,45000
129900,129900
422900,422900
422900,422900
290000,290000


In [63]:
-- Convert Sale price to int avoid decimal places
Alter TABLE NashvilleHousing
Alter COLUMN SalePrice bigint; 

Commands completed successfully.

Total execution time: 00:00:00.009

In [64]:
select top 5 SalePrice from NashvilleHousing;

(5 rows affected)

Total execution time: 00:00:00.006

SalePrice
45000
129900
422900
422900
290000


## **Data Cleaning/Imputation**

### **Imputing Property Address Values**

Some property addresses have NULL addresses but shares the same parcelId with other properties where the property Address field is populated. So these NULL property addresses can be updated using the corresponding records that has the same parcelDs.

Here is the algorithm used

1. Create a Common Table Expression for finding the previous and next addresses and parcelDs partitioned on pracelIDs
2. Find the addresses that has null in property address which has property address populated in the next or previous row while having the same parcelID
3. Update the address field using the previous or next address

In [65]:
with propAddCTE as (select parcelId
, propertyAddress
, LAG(propertyAddress) over (PARTITION by parcelId order by parcelId) as PrevPropertyAddress
, LEAD(PropertyAddress) over (partition by parcelId order by parcelId) as NextPropertyAddress
, LAG(ParcelID) over (partition by parcelId order by parcelId) as PrevParcelId
, LEAD(ParcelID) over (partition by parcelId order by parcelId) as NextPracelId
from NashvilleHousing)

select top 5 * from propAddCTE
where propertyAddress is null
and (((ParcelID = PrevParcelId) and (PrevPropertyAddress is not null) ) 
        or ((ParcelID = NextPracelId) and (NextPropertyAddress is not null)))

(5 rows affected)

Total execution time: 00:00:00.176

parcelId,propertyAddress,PrevPropertyAddress,NextPropertyAddress,PrevParcelId,NextPracelId
025 07 0 031.00,NULL,"410 ROSEHILL CT, GOODLETTSVILLE",NULL,025 07 0 031.00,NULL
026 01 0 069.00,NULL,"141 TWO MILE PIKE, GOODLETTSVILLE",NULL,026 01 0 069.00,NULL
026 05 0 017.00,NULL,"208 EAST AVE, GOODLETTSVILLE",NULL,026 05 0 017.00,NULL
033 06 0A 002.00,NULL,"1116 CAMPBELL RD, GOODLETTSVILLE",NULL,033 06 0A 002.00,NULL
034 03 0 059.00,NULL,"2117 PAULA DR, MADISON","2117 PAULA DR, MADISON",034 03 0 059.00,034 03 0 059.00


In [66]:
-- Checking a Sample ParcelId
select * from NashvilleHousing where ParcelID = '026 06 0A 038.00'

(2 rows affected)

Total execution time: 00:00:00.032

UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,OwnerName,OwnerAddress,Acreage,TaxDistrict,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath
19828,026 06 0A 038.00,RESIDENTIAL CONDO,"109 CANTON CT, GOODLETTSVILLE",2014-08-21,127000,20140825-0077233,No,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
53147,026 06 0A 038.00,RESIDENTIAL CONDO,NULL,2016-08-25,144900,20160831-0091567,No,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


Check how many records are there with null as property address

In [67]:
select count(*) as NullAddressCount from NashvilleHousing where PropertyAddress is null

(1 row affected)

Total execution time: 00:00:00.038

NullAddressCount
29


Update first set of addresses

In [68]:
with propAddCTE as (select parcelId
, propertyAddress
, LAG(propertyAddress) over (PARTITION by parcelId order by parcelId) as PrevPropertyAddress
, LEAD(PropertyAddress) over (partition by parcelId order by parcelId) as NextPropertyAddress
, LAG(ParcelID) over (partition by parcelId order by parcelId) as PrevParcelId
, LEAD(ParcelID) over (partition by parcelId order by parcelId) as NextParcelId
from NashvilleHousing)
-- Update addresses
Update NashvilleHousing 
set propertyAddress = pa.PrevPropertyAddress 
from propAddCTE pa 
join NashvilleHousing nh on nh.ParcelID = pa.parcelId
where pa.parcelId = nh.parcelId and (pa.parcelId = pa.PrevParcelId)
and nh.PropertyAddress is null and pa.PrevPropertyAddress is not null 

(28 rows affected)

Total execution time: 00:00:00.142

In [69]:
select count(*) as NullAddressCount from NashvilleHousing where PropertyAddress is null

(1 row affected)

Total execution time: 00:00:00.026

NullAddressCount
1


Update the second set of addresses

In [70]:
with propAddCTE as (select parcelId
, propertyAddress
, LAG(propertyAddress) over (PARTITION by parcelId order by parcelId) as PrevPropertyAddress
, LEAD(PropertyAddress) over (partition by parcelId order by parcelId) as NextPropertyAddress
, LAG(ParcelID) over (partition by parcelId order by parcelId) as PrevParcelId
, LEAD(ParcelID) over (partition by parcelId order by parcelId) as NextParcelId
from NashvilleHousing)
-- Update addresses
Update NashvilleHousing 
set propertyAddress = pa.NextPropertyAddress 
from propAddCTE pa 
join NashvilleHousing nh on nh.ParcelID = pa.parcelId
where pa.parcelId = nh.parcelId and (pa.parcelId = pa.NextParcelId)
and nh.PropertyAddress is null and pa.NextPropertyAddress is not null 

(1 row affected)

Total execution time: 00:00:00.112

In [71]:
select count(*) as NullAddressCount from NashvilleHousing where PropertyAddress is null

(1 row affected)

Total execution time: 00:00:00.046

NullAddressCount
0


### **Property Address Split**

Split the property address field into Street and City

In [77]:
select top 10 propertyAddress,
 substring(propertyAddress,1,CHARINDEX(',',PropertyAddress)-1) as PropertyStreet,
 substring(PropertyAddress,CHARINDEX(',',PropertyAddress)+1,len(PropertyAddress)) as PropertyCity
from NashvilleHousing

(10 rows affected)

Total execution time: 00:00:00.013

propertyAddress,StreetName,City
"134 HAMBLEN AVE, OLD HICKORY",134 HAMBLEN AVE,OLD HICKORY
"138 HAMBLEN AVE, OLD HICKORY",138 HAMBLEN AVE,OLD HICKORY
"800 CERRITO LNDG, OLD HICKORY",800 CERRITO LNDG,OLD HICKORY
"804 CERRITO LNDG, OLD HICKORY",804 CERRITO LNDG,OLD HICKORY
"808 CERRITO LNDG, OLD HICKORY",808 CERRITO LNDG,OLD HICKORY
"812 CERRITO LNDG, OLD HICKORY",812 CERRITO LNDG,OLD HICKORY
"608 LAKEMEADE PT, OLD HICKORY",608 LAKEMEADE PT,OLD HICKORY
"520 NAWAKWA TRL, MADISON",520 NAWAKWA TRL,MADISON
"1524 JOE PYRON DR, MADISON",1524 JOE PYRON DR,MADISON
"1516 JOE PYRON DR, MADISON",1516 JOE PYRON DR,MADISON


In [78]:
-- Add a new column '[PropertyStreet]', [PropertyCity] to table '[NashvilleHousing]' in schema '[dbo]'
ALTER TABLE [dbo].[NashvilleHousing]
    ADD [PropertyStreet] varchar(100)  NULL,
        [PropertyCity] varchar(100) NULL
GO

Commands completed successfully.

Total execution time: 00:00:00.077

In [79]:
-- Update rows in table '[NashvilleHousing]' in schema '[dbo]'
UPDATE [dbo].[NashvilleHousing]
SET
    [PropertyStreet] = substring(propertyAddress,1,CHARINDEX(',',PropertyAddress)-1),
    [PropertyCity] = substring(PropertyAddress,CHARINDEX(',',PropertyAddress)+1,len(PropertyAddress))
GO

(56477 rows affected)

Total execution time: 00:00:00.654

In [84]:
select top 10 propertyAddress, PropertyStreet, PropertyCity from NashvilleHousing

(10 rows affected)

Total execution time: 00:00:00.023

propertyAddress,propertyStreet,propertyCity
"134 HAMBLEN AVE, OLD HICKORY",134 HAMBLEN AVE,OLD HICKORY
"138 HAMBLEN AVE, OLD HICKORY",138 HAMBLEN AVE,OLD HICKORY
"800 CERRITO LNDG, OLD HICKORY",800 CERRITO LNDG,OLD HICKORY
"804 CERRITO LNDG, OLD HICKORY",804 CERRITO LNDG,OLD HICKORY
"808 CERRITO LNDG, OLD HICKORY",808 CERRITO LNDG,OLD HICKORY
"812 CERRITO LNDG, OLD HICKORY",812 CERRITO LNDG,OLD HICKORY
"608 LAKEMEADE PT, OLD HICKORY",608 LAKEMEADE PT,OLD HICKORY
"520 NAWAKWA TRL, MADISON",520 NAWAKWA TRL,MADISON
"1524 JOE PYRON DR, MADISON",1524 JOE PYRON DR,MADISON
"1516 JOE PYRON DR, MADISON",1516 JOE PYRON DR,MADISON


### **Owner Address Split using PARSENAME**

PARSENAME command parses a string using '.' or periods. So, as a first step we should convert all commas to periods before using PARSENAME. Note: The function parses a string from right to left.

In [87]:
select top 5 ownerAddress
, PARSENAME(REPLACE(ownerAddress,',','.'),3) as OwnerStreet
, PARSENAME(REPLACE(ownerAddress,',','.'),2) as OwnerCity
, PARSENAME(REPLACE(ownerAddress,',','.'),1) as OwnerState
from NashvilleHousing

(5 rows affected)

Total execution time: 00:00:00.033

ownerAddress,OwnerStreet,OwnerCity,OwnerState
"134 HAMBLEN AVE, OLD HICKORY, TN",134 HAMBLEN AVE,OLD HICKORY,TN
"138 HAMBLEN AVE, OLD HICKORY, TN",138 HAMBLEN AVE,OLD HICKORY,TN
"800 CERRITO LNDG, OLD HICKORY, TN",800 CERRITO LNDG,OLD HICKORY,TN
"804 CERRITO LNDG, OLD HICKORY, TN",804 CERRITO LNDG,OLD HICKORY,TN
"808 CERRITO LNDG, OLD HICKORY, TN",808 CERRITO LNDG,OLD HICKORY,TN


In [89]:
-- Add new columns '[OwnerStreet]', '[OwnerCity]' and '[OwnerState]' to table '[NashvilleHousing]' in schema '[dbo]'
ALTER TABLE [dbo].[NashvilleHousing]
    ADD [OwnerStreet] varchar(100)  NULL,
        [OwnerCity] varchar(100) NULL,
        [OwnerState] varchar(25) NULL
GO

Commands completed successfully.

Total execution time: 00:00:00.065

In [90]:
-- Update rows in table '[NashvilleHousing]' in schema '[dbo]'
UPDATE [dbo].[NashvilleHousing]
SET
    [OwnerStreet]   = PARSENAME(REPLACE(ownerAddress,',','.'),3),
    [OwnerCity]     = PARSENAME(REPLACE(ownerAddress,',','.'),2),
    [OwnerState]    = PARSENAME(REPLACE(ownerAddress,',','.'),1)
GO

(56477 rows affected)

Total execution time: 00:00:00.512

In [91]:
select top 5 ownerAddress
, OwnerStreet
, OwnerCity
, OwnerState
from NashvilleHousing

(5 rows affected)

Total execution time: 00:00:00.031

ownerAddress,OwnerStreet,OwnerCity,OwnerState
"134 HAMBLEN AVE, OLD HICKORY, TN",134 HAMBLEN AVE,OLD HICKORY,TN
"138 HAMBLEN AVE, OLD HICKORY, TN",138 HAMBLEN AVE,OLD HICKORY,TN
"800 CERRITO LNDG, OLD HICKORY, TN",800 CERRITO LNDG,OLD HICKORY,TN
"804 CERRITO LNDG, OLD HICKORY, TN",804 CERRITO LNDG,OLD HICKORY,TN
"808 CERRITO LNDG, OLD HICKORY, TN",808 CERRITO LNDG,OLD HICKORY,TN


### **SoldAsVacant Field Update**

The SoldAsVacant field is a field that is supposed to have only 'Yes' and 'No' values. Let us take a look at the column.

In [2]:
select DISTINCT SoldasVacant
, count(*) as count 
from NashvilleHousing
group by SoldAsVacant
order by 2

(4 rows affected)

Total execution time: 00:00:00.047

SoldasVacant,count
Y,52
N,399
Yes,4623
No,51403


Changing all 'Y' to 'Yes' and 'N' to 'No's

In [7]:
-- Update rows in table '[TableName]' in schema '[dbo]'
UPDATE [dbo].[NashvilleHousing]
SET
    [SoldAsVacant] = CASE   WHEN SoldAsVacant = 'Y' THEN 'Yes'
                            WHEN SoldAsVacant = 'N' THEN 'No'
                            ELSE SoldAsVacant
                     END
GO 

(56477 rows affected)

Total execution time: 00:00:00.186

In [8]:
select DISTINCT SoldasVacant
, count(*) as count 
from NashvilleHousing
group by SoldAsVacant
order by 2

(2 rows affected)

Total execution time: 00:00:00.163

SoldasVacant,count
Yes,4675
No,51802


### **Remove Duplicates**

The table has some properties that are duplicates. We are considering those properties that has same <span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">ParcelID, propertyAddress, SaleDate, legalReference as duplicates</span>

In [18]:
select top 5 * from (select ParcelID, propertyAddress, SaleDate, legalReference, count(*) as cnt from NashvilleHousing
group by ParcelID, propertyAddress, SaleDate,legalReference) fq
where  cnt > 1

(5 rows affected)

Total execution time: 00:00:00.107

ParcelID,propertyAddress,SaleDate,legalReference,cnt
081 07 0 265.00,"1806 15TH AVE N, NASHVILLE",2015-02-17,20150223-0015122,2
081 11 0 168.00,"1710 DR D B TODD JR BLVD, NASHVILLE",2015-02-13,20150218-0013602,2
081 11 0 495.00,"1718 ARTHUR AVE, NASHVILLE",2015-02-09,20150210-0012450,2
081 15 0 472.00,"1818 B SCOVEL ST, NASHVILLE",2015-02-20,20150223-0015257,2
081 15 0 263.00,"1520 14TH AVE N, NASHVILLE",2015-02-12,20150218-0013742,2


We are deleting rows using Common Table Expression.

In [28]:
WITH DuplicateRowsCTE as (
    select uniqueID,
            ROW_NUMBER() over (partition by ParcelID, propertyAddress, SaleDate, legalReference order by uniqueID) rownum 
            from NashvilleHousing
)

-- Delete rows from table '[NashvilleHousing]' in schema '[dbo]'
DELETE FROM [dbo].[NashvilleHousing]
WHERE UniqueID in (select UniqueID from DuplicateRowsCTE
where rownum >1)
GO 

(104 rows affected)

Total execution time: 00:00:00.149

### **Delete Unwanted Columns**

In [30]:
ALTER TABLE NashvilleHousing
drop column TaxDistrict
GO

Commands completed successfully.

Total execution time: 00:00:00.093